In [33]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "Userpass"
animals = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(animals.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A([
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
             alt='image', style={'height':'10%', 'width':'10%'})], href='https://www.snhu.edu'),
    
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.H2('Jon Curtis')),
    html.Hr(),
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
               {'label': 'Water Rescue', 'value': 1},
               {'label': 'Mountain Rescue', 'value': 2},
               {'label': 'Disaster Rescue', 'value': 3},
               {'label': 'Reset','value': 4}
            ],
            value='Reset',
            labelStyle={'display': 'inline-block'}
        ) 
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        fixed_columns={'headers': True, 'data': 1},
        style_table={'minWidth': '100%'},
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10, 
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(value):
    
    if (value==1):
        df = pd.DataFrame(animals.read({"animal_type":"Dog",
        "breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
        "sex_upon_outcome":"Intact Female",
        "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}))
    elif (value==2):
        df = pd.DataFrame(animals.read({"animal_type":"Dog",
        "breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
        "sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}))
    elif (value==3):
        df = pd.DataFrame(animals.read({"animal_type":"Dog",
        "breed":{"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},
        "sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}}))
    else:
        df = pd.DataFrame.from_records(animals.read({}))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return (data,columns)


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    if viewData is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure = px.pie(dff, names = 'breed')
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', "selected_rows"),
     Input('datatable-id', "selected_columns")])
def update_map(viewData, selected_rows, selected_columns):
    dff = pd.DataFrame.from_dict(viewData)
    if selected_rows == []:
        selected_rows = [0]
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[(dff.iloc[selected_rows[0],13]), (dff.iloc[selected_rows[0],14])], children=[
                dl.Tooltip(dff.iloc[selected_rows[0],4]),
                dl.Popup([
                    html.H1('Animal Name'),
                    html.P(dff.iloc[selected_rows[0],9])
                ])
            ])
        ])
    ]
app